In [1]:
# -----------------------------
# Mount Google Drive
# -----------------------------
from google.colab import drive
drive.mount('/content/drive')

# -----------------------------
# Imports
# -----------------------------
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset
import torch

# -----------------------------
# Model & tokenizer
# -----------------------------
MODEL_NAME = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

# -----------------------------
# Dataset
# -----------------------------
with open("fairy_tales_cleaned.txt", "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

class StoryDataset(Dataset):
    def __init__(self, texts, tokenizer, block_size=256):
        self.examples = []
        for story in texts:
            tokenized = tokenizer(
                story,
                truncation=True,
                max_length=block_size,
                padding="max_length",
                return_tensors="pt"
            )
            self.examples.append(tokenized["input_ids"].squeeze())

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

train_dataset = StoryDataset(lines, tokenizer, block_size=256)

# -----------------------------
# Data collator
# -----------------------------
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# -----------------------------
# Output directory on Google Drive
# -----------------------------
output_dir = "/content/drive/MyDrive/gpt2-fairytales-medium"

# -----------------------------
# Training arguments (optimized for T4)
# -----------------------------
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=20,                # train longer for better fine-tuning
    per_device_train_batch_size=1,      # small batch to fit T4
    gradient_accumulation_steps=8,      # simulate effective batch size of 8
    save_steps=200,                     # frequent checkpointing
    save_total_limit=5,                 # keep last 5 checkpoints
    logging_steps=50,
    learning_rate=5e-5,
    fp16=True,                           # mixed precision for speed & memory
    prediction_loss_only=True
)

# -----------------------------
# Trainer
# -----------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# -----------------------------
# Start training
# -----------------------------
trainer.train()

# -----------------------------
# Save final model & tokenizer
# -----------------------------
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Training complete! Model saved to Google Drive.")


Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nipunikumudika (nipunikumudika-home) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Step,Training Loss
50,3.037100
100,2.966600
150,2.913500
200,2.877100
250,2.650300
300,2.667800
350,2.631800
400,2.634000
450,2.470100
500,2.419400


Training complete! Model saved to Google Drive.
